# 텍스트 (지시) 완료 클라이언트 사용 방법

EasyLLM은 `text-davinci-003`을 오픈 소스 모델로 대체하는 추상화 계층으로 사용할 수 있습니다.

클라이언트만 변경하여 OpenAI API에서 자체 애플리케이션을 변경할 수 있습니다.

채팅 모델은 일련의 메시지를 입력으로 사용하고 AI가 작성한 메시지를 출력으로 반환합니다.

이 가이드에서는 몇 가지 예제 API 호출을 통해 채팅 형식을 보여줍니다.


## 0. 설정

Amazon SageMaker에서 `easyllm`을 사용하려면 먼저 모델을 SageMaker 엔드포인트에 배포해야 합니다. 아래 블로그 게시물 중 하나를 따라 이 작업을 수행할 수 있습니다.

* [Amazon SageMaker에 Llama 2 7B/13B/70B 배포](https://www.philschmid.de/sagemaker-llama-llm)
* [Amazon SageMaker에 Falcon 7B & 40B 배포](https://www.philschmid.de/sagemaker-falcon-llm)
* [Amazon SageMaker용 Hugging Face LLM 추론 컨테이너 소개](https://www.philschmid.de/sagemaker-huggingface-llm)

엔드포인트 배포가 완료되면 엔드포인트 이름을 복사합니다. 엔드포인트 이름이 `model` 매개변수가 됩니다. AWS 관리 콘솔의 Amazon SageMaker에서 "추론" -> "엔드포인트" -> "이름"으로 이동하거나 sagemaker SDK를 사용하여 모델을 배포한 경우 `predictor.endpoint_name` 속성에서 엔드포인트 이름을 가져올 수 있습니다.

### 1. easyllm 라이브러리 가져오기

In [ ]:
# 필요한 경우 OpenAI Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm 

In [6]:
# EasyLLM API를 호출하기 위한 EasyLLM Python 라이브러리를 가져옵니다.
import easyllm

### 2. 채팅 API 호출 예제

텍스트 API 호출에는 두 가지 필수 입력이 있습니다.
- `model`: 사용하려는 모델의 이름(예: `meta-llama/Llama-2-70b-chat-hf`) 또는 API만 호출하려면 비워 둡니다.
- `prompt`: 텍스트를 생성하기 위해 모델로 전송되는 텍스트 프롬프트

OpenAI API와 비교하여 `huggingface` 모듈은 프롬프트 및 중지 시퀀스를 사용자 지정하는 데 사용할 수 있는 `prompt_builder` 및 `stop_sequences` 매개변수도 노출합니다. EasyLLM 패키지에는 이러한 두 매개변수에 대한 널리 사용되는 메서드(예: `llama2_prompt_builder` 및 `llama2_stop_sequences`)가 내장되어 있습니다.

채팅 형식이 실제로 어떻게 작동하는지 확인하기 위해 예제 채팅 API 호출을 살펴보겠습니다.

In [1]:
import os 
# 프롬프트 빌더용 환경 변수 설정
os.environ["HUGGINGFACE_PROMPT"] = "llama2" # vicuna, wizardlm, stablebeluga, open_assistant
os.environ["AWS_REGION"] = "us-east-1"  # 해당 지역으로 변경
# os.environ["AWS_ACCESS_KEY_ID"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요
# os.environ["AWS_SECRET_ACCESS_KEY"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요

from easyllm.clients import sagemaker

# 환경 변수를 사용하지 않고 구성 변경
# sagemaker.prompt_builder = "llama2"
# sagemaker.api_aws_access_key="xxx"
# sagemaker.api_aws_secret_key="xxx"

# SageMaker 엔드포인트 이름
MODEL="huggingface-pytorch-tgi-inference-2023-08-08-14-15-52-703"

response = sagemaker.Completion.create(
    model=MODEL,
    prompt="삶의 의미는 무엇인가요?",
    temperature=0.9,
    top_p=0.6,
    max_tokens=256,
)
response

{'id': 'hf-dEMeXTUk3Y',
 'object': 'text.completion',
 'created': 1691508711,
 'model': 'huggingface-pytorch-tgi-inference-2023-08-08-14-15-52-703',
 'choices': [{'index': 0,
   'text': " 삶의 의미는 역사상 철학자, 신학자, 과학자 및 기타 많은 사상가들을 당혹스럽게 만든 질문입니다. 다음은 몇 가지 가능한 답변입니다.\n1. 종교적 또는 영적 신념: 많은 사람들은 삶의 의미가 도덕적 계율을 따르거나, 영적 깨달음을 얻거나, 특정 사명이나 소명을 완수하는 등 신성하거나 영적인 목적을 달성하는 것이라고 믿습니다.\n2. 개인적 성취: 어떤 사람들은 삶의 의미가 개인적인 성취와 행복을 찾는 것이라고 믿습니다. 이것은 자신의 열정을 추구하고, 의미 있는 관계를 구축하고, 삶의 목적과 의미를 함양함으로써 달성할 수 있습니다.\n3. 사회에 대한 기여: 많은 사람들은 삶의 의미가 세상에 긍정적인 영향을 미치고 더 큰 선에 기여하는 것이라고 믿습니다. 이것은 세상을 더 나은 곳으로 만들기 위해 노력하거나, 다른 사람을 돕거나, 가치 있는 것을 창조하는 등 다양한 수단을 통해 달성할 수 있습니다.\n4. 학습과 성장: 어떤 사람들은 삶의 의미가 개인으로서 배우고 성장하고, 세상에 대한 지식과 이해를 넓히고, 기술을 개발하는 것이라고 믿습니다",
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 11, 'completion_tokens': 256, 'total_tokens': 267}}

보시다시피 응답 개체에는 몇 가지 필드가 있습니다.
- `id`: 요청 ID
- `object`: 반환된 개체 유형(예: `text.completion`)
- `created`: 요청 타임스탬프
- `model`: 응답을 생성하는 데 사용된 모델의 전체 이름
- `usage`: 프롬프트, 완성 및 합계를 계산하여 회신을 생성하는 데 사용된 토큰 수
- `choices`: 완성 개체 목록(`n`을 1보다 크게 설정하지 않은 경우 하나만)
    - `text`: 생성된 텍스트
    - `finish_reason`: 모델이 텍스트 생성을 중지한 이유(`stop`, `eos_token` 또는 `max_tokens` 제한에 도달한 경우 `length`)
    - `logprobs`: _선택 사항_ 생성된 각 토큰의 로그 확률입니다.

다음을 사용하여 회신만 추출합니다.

In [2]:
print(response['choices'][0]['text'])

 삶의 의미는 역사상 철학자, 신학자, 과학자 및 기타 많은 사상가들을 당혹스럽게 만든 질문입니다. 다음은 몇 가지 가능한 답변입니다.
1. 종교적 또는 영적 신념: 많은 사람들은 삶의 의미가 도덕적 계율을 따르거나, 영적 깨달음을 얻거나, 특정 사명이나 소명을 완수하는 등 신성하거나 영적인 목적을 달성하는 것이라고 믿습니다.
2. 개인적 성취: 어떤 사람들은 삶의 의미가 개인적인 성취와 행복을 찾는 것이라고 믿습니다. 이것은 자신의 열정을 추구하고, 의미 있는 관계를 구축하고, 삶의 목적과 의미를 함양함으로써 달성할 수 있습니다.
3. 사회에 대한 기여: 많은 사람들은 삶의 의미가 세상에 긍정적인 영향을 미치고 더 큰 선에 기여하는 것이라고 믿습니다. 이것은 세상을 더 나은 곳으로 만들기 위해 노력하거나, 다른 사람을 돕거나, 가치 있는 것을 창조하는 등 다양한 수단을 통해 달성할 수 있습니다.
4. 학습과 성장: 어떤 사람들은 삶의 의미가 개인으로서 배우고 성장하고, 세상에 대한 지식과 이해를 넓히고, 기술을 개발하는 것이라고 믿습니다
